In [2]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization, GRU, Masking
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
# from numpy import argmax
# from pickle import load
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
# # from nltk.translate.bleu_score import corpus_bleu
import json
# import random
import csv
from keras.regularizers import l2, l1
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import tensorflow as tf

import os.path as osp
import os
from pprint import pprint
from skimage.transform import rescale, resize
from skimage import data, color, io
import skimage
import PIL
import scipy
import json
import os.path
from os import path

from keras.applications.xception import preprocess_input
from keras.applications.xception import Xception
from keras.models import Model
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import decode_predictions



#Wesam
# SEED = 10
#IMAGE_EMBEDDING_VAL_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/validation/'
# IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/one_sample_cnn/'
# filepath = '/content/drive/My Drive/Colab_Notebooks/DL_data/model-ep{epoch:03d}-loss{loss:.3f}.h5'
#CAPTION_FILE_NAME = '/content/drive/My Drive/Colab_Notebooks/DL_data/image_to_caption.csv'



#on my Colab
ALL_CAPTIONS_FILE = '/content/drive/My Drive/Colab_Notebooks/DL_data/all_captions.txt'
COMPLETE_STORIES_FILE = '/content/drive/My Drive/Colab_Notebooks/DL_data/complete_stories_all_splits.json'
IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/CNNFeatureVectors/'
GLOVE_EMBEDDING_FILE_NAME = '/content/drive/My Drive/Colab_Notebooks/DL_data/glove.6B.300d.txt'


# #For my GCP:
# ALL_CAPTIONS_FILE = 'all_captions.txt'
# COMPLETE_STORIES_FILE = 'complete_stories_all_splits.json'
# IMAGE_EMBEDDING_DIR = 'CNNFeatureVectors/'
# GLOVE_EMBEDDING_FILE_NAME = 'glove.6B.300d.txt'


#Vinuta
SEED = 10
IMAGE_EMBEDDING_DIM = 2048
#IMAGE_EMBEDDING_DIR = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/'
NUM_IMAGE_EMBEDDING_CHUNKS = 10
#GLOVE_EMBEDDING_FILE_NAME = 'glove.6B.300d.txt'
MAX_SEQUENCE_LENGTH = 11
WORD_EMBEDDING_DIM = 300
#CAPTION_FILE_NAME = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/image_to_caption.csv'
SENTENCE_EMBEDDING_DIM = 512
VOCAB_SIZE=26572 ## must change to 26572

#commented for GCP
from google.colab import drive
drive.mount('/content/drive')

embedding_matrix = np.load('/content/drive/My Drive/Colab_Notebooks/DL_data/embedding_matrix.npy')

with open('/content/drive/My Drive/Colab_Notebooks/DL_data/ixtoword.json', 'r') as fp:
    ixtoword =  json.load(fp)

with open('/content/drive/My Drive/Colab_Notebooks/DL_data/wordtoix.json', 'r') as fp:
    wordtoix =  json.load(fp)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#TMP:


# embedding_matrix = np.delete(embedding_matrix, 0, 0)

#Create model

In [3]:
#custom loss function
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred)
    loss_mean = tf.reduce_mean(loss)
    return loss_mean


#restructure
#img_layer
inputs1 = Input(shape=(5,IMAGE_EMBEDDING_DIM),name='inputs1_layer')
#inputs1 = Input(shape=(5,),name='inputs1_layer')
#img1 = Embedding(len(image_embd), IMAGE_EMBEDDING_DIM , mask_zero=True, weights=[img_embedding_matrix], trainable=False, name='img1_layer')(inputs1) #toChange
img2 = Dropout(0.5, name='img2_layer')(inputs1)
img3 , hidden_img = GRU(300, recurrent_dropout=0, return_state=True , activity_regularizer=l2(0.001), name='img3_layer')(img2)

#prev_sentences_layer
inputs3 = Input(shape=(MAX_SEQUENCE_LENGTH*4,), name='inputs3_layer')
sen1= Embedding(VOCAB_SIZE, WORD_EMBEDDING_DIM, mask_zero=True, weights=[embedding_matrix], trainable=False, name='sen1_layer')(inputs3)
sen2 = Dropout(0.5, name='sen2_layer')(sen1)
sen3,sen_hidden_f , sen_hidden_b = Bidirectional(GRU(300,recurrent_dropout=0, return_state=True, activity_regularizer=l2(0.001)), merge_mode='sum', name='sen3_layer')(sen2)
hidden_sen =  add([sen_hidden_f,sen_hidden_b], name='hidden_sen_layer')
decoder_hidden = add([hidden_img,hidden_sen], name='decoder_hidden_layer')

#Current caption layer
inputs2 = Input(shape=(MAX_SEQUENCE_LENGTH,),name='inputs2_layer')
seq1 = Embedding(VOCAB_SIZE, WORD_EMBEDDING_DIM, mask_zero=True, weights=[embedding_matrix], trainable=False, name='seq1_layer')(inputs2)
seq2 = Dropout(0.5, name='seq2_layer')(seq1)
decoder1 = GRU(300,recurrent_dropout=0, return_sequences=True ,activity_regularizer=l2(0.001), name='decoder1_layer')(seq2, initial_state=decoder_hidden)


decoder2 = Dense(300, activation=None, kernel_regularizer=l2(0.001), name='decoder2_layer')(decoder1)
outputs = Dense(VOCAB_SIZE, activation='linear', name='outputs_layer')(decoder2) ##was softmax /// used linear because it's recommended with the custom loss: https://github.com/tensorflow/tensorflow/issues/17150
model_inference = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)

model_inference.summary()
decoder_target = tf.placeholder(dtype='int32', shape=(None, None))
model_inference.compile(loss=sparse_cross_entropy, optimizer='adam', target_tensors=[decoder_target])#'adam')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs3_layer (InputLayer)      (None, 44)           0                                            
__________________________________________________________________________________________________
sen1_layer (Embedding)          (None, 44, 300)      7971600     inputs3_layer[0][0]              
__________________________________________________________________________________________________
inputs1_layer (InputLayer)      (None, 5, 2048)      0                                            
_______________________________________________________________________

#Load weights

In [0]:
model_inference.load_weights('/content/drive/My Drive/Colab_Notebooks/DL_data/models/final_model_allstories_seqlen11_weights.h5', by_name=True)

In [0]:
names = [weight.name for layer in model_inference.layers for weight in layer.weights]
weights = model_inference.get_weights()

for name, weight in zip(names, weights):
    print(name, weight)

In [0]:


def load_image(image_path,target_size):
    img = skimage.io.imread(image_path)
    image_resized = skimage.transform.resize(img, target_size, anti_aliasing=True)
    return image_resized

def load_cnn_model():
    model = Xception()
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-1].output)
    return model

def extract_features_from_images(image_path):
    model = load_cnn_model()
    if path.exists(image_path):
        print(image_path)
        image = load_image(image_path, target_size=(299, 299))
        if image.shape == (299, 299, 3):
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = preprocess_input(image)
            feature = model.predict(image, verbose=0)
            print(feature)
            return feature


#########################################
# def generate_desc(model, photo, max_length):
#     in_text = 'startseq'
#     for i in range(max_length):
#         sequence = [wordtoix[word] for word in in_text.split() if word in wordtoix]
#         sequence = pad_sequences([sequence], maxlen=max_length)
#         yhat = model.predict([photo,sequence], verbose=0)
#         yhat = np.argmax(yhat)
#         # map integer to word
#         word = ixtoword[yhat]
#         if word is None:
#             break
#         in_text += ' ' + word
#         if word == 'endseq':
#             break
#     return in_text

#########################################

def generate_desc(model, photos, max_length):
    prev_sents='' 
    for counter in range(5):
      #print(prev_sents)
      in_text = 'startseq'
      img_seq = photos[:counter+1] #get_empty_feature()
      img_seq = padd_image_embed_seq(img_seq)

      prev_embeddings = [int(wordtoix[word]) for word in prev_sents.split() if word in wordtoix]
      prev_embeddings = pad_sequences([prev_embeddings], maxlen=max_length*4)[0]
      for i in range(max_length):
        seq_in = [int(wordtoix[word]) for word in in_text.split() if word in wordtoix]
        seq_in = pad_sequences([seq_in], maxlen=MAX_SEQUENCE_LENGTH, padding='post')[0]

        yhat = model_inference.predict([img_seq[np.newaxis, ...],seq_in[np.newaxis, ...], prev_embeddings[np.newaxis, ...]], verbose=0)

        yhat = yhat[0,i,:]
        yhat = np.argmax(yhat)

        # map integer to word
        #word = ixtoword[yhat]
        cur_word = None
        if i+1==max_length:
          prev_sents += ' ' + in_text.split(' ', 1)[1]
          break
        if str(yhat) in ixtoword:
          cur_word = ixtoword[str(yhat)]#[ixtoword[word] for word in yhat if word in ixtoword]
          if cur_word != 'endseq':
            in_text += ' ' + cur_word
          else:
            prev_sents += ' ' + in_text.split(' ', 1)[1]
            break
        else:
            continue
      #print("prev_sentes are:")
      #print(prev_sents)
      #print("------------------------")
    return prev_sents

#Prediction

In [0]:
def get_empty_feature():
  return np.zeros(IMAGE_EMBEDDING_DIM)

#np.append(A, [[7, 8, 9]], axis=0)
def padd_image_embed_seq(img_seq):
  if np.shape(img_seq)[1] < 2048:
    img_seq = [get_empty_feature()]
  to_add = 5 - np.shape(img_seq)[0]
  for i in range(to_add):
    img_seq = np.insert(img_seq, 0 ,get_empty_feature(),  axis=0)##np.append(get_empty_feature() , img_seq, axis=0)####
  return img_seq

In [0]:
# img_sequence=[np.ones(IMAGE_EMBEDDING_DIM)]
# print(np.shape(img_sequence)[0])
# print(img_sequence)
# img_sequence = padd_image_embed_seq(img_sequence)
# print(np.shape(img_sequence))
# print(img_sequence)


In [57]:

image_path1='/content/drive/My Drive/Colab_Notebooks/DL_data/images/1.jpg'#'/Users/vinutahegde/Documents/Personal/IMG_3501.JPG'
image_path2='/content/drive/My Drive/Colab_Notebooks/DL_data/images/2.jpg'
image_path3='/content/drive/My Drive/Colab_Notebooks/DL_data/images/3.jpg'
image_path4='/content/drive/My Drive/Colab_Notebooks/DL_data/images/4.jpg'
image_path5='/content/drive/My Drive/Colab_Notebooks/DL_data/images/5.jpg'#/from_train_not_a_story


feature1 = extract_features_from_images(image_path1)
feature2 = extract_features_from_images(image_path2)
feature3 = extract_features_from_images(image_path3)
feature4 = extract_features_from_images(image_path4)
feature5 = extract_features_from_images(image_path5)

features=np.concatenate((feature1,feature2,feature3,feature4,feature5), axis=0)

/content/drive/My Drive/Colab_Notebooks/DL_data/images/1.jpg
[[0.         0.         0.19784988 ... 0.1310169  0.         0.        ]]
/content/drive/My Drive/Colab_Notebooks/DL_data/images/2.jpg
[[0.         0.         0.15794557 ... 0.13588832 0.         0.        ]]
/content/drive/My Drive/Colab_Notebooks/DL_data/images/3.jpg
[[0.         0.         0.11628581 ... 0.112457   0.         0.        ]]
/content/drive/My Drive/Colab_Notebooks/DL_data/images/4.jpg
[[0.         0.         0.16458069 ... 0.12686986 0.         0.        ]]
/content/drive/My Drive/Colab_Notebooks/DL_data/images/5.jpg
[[0.         0.         0.29422185 ... 0.15950316 0.         0.        ]]


In [32]:
print(np.shape(features))


(5, 2048)


In [58]:
#features_empty=np.concatenate(([get_empty_feature()],[get_empty_feature()],[get_empty_feature()],[get_empty_feature()],[get_empty_feature()]), axis=0)
#print(np.shape(features_empty))
print(generate_desc(model_inference, features, MAX_SEQUENCE_LENGTH))

 the family went to the party and the kids were having fun and the kids were having a great time and the kids were having a great time and then they were having a great time


In [57]:
print(ixtoword['5'])

{'1': 'startseq', '2': 'endseq', '3': 'my', '4': 'trip', '5': 'to', '6': 'location', '7': 'was', '8': 'amazing', '9': '.', '10': 'i', '11': 'saw', '12': 'some', '13': 'colorful', '14': 'people', '15': 'even', '16': 'made', '17': 'knew', '18': 'friends', '19': 'visited', '20': 'a', '21': 'beautiful', '22': 'pagoda', '23': 'then', '24': 'ordinary', '25': 'bicycle', '26': 'there', '27': 'is', '28': 'little', '29': 'place', '30': 'in', '31': 'the', '32': 'city', '33': 'called', '34': 'china', '35': 'lot', '36': 'of', '37': 'make', '38': 'it', '39': 'feel', '40': 'like', '41': 'you', '42': 'are', '43': 'actually', '44': 'come', '45': 'and', '46': 'visit', '47': 'take', '48': 'pictures', '49': 'with', '50': 'chinese', '51': 'structures', '52': 'cultural', '53': 'everyone', '54': 'has', '55': 'their', '56': 'own', '57': 'way', '58': 'celebrating', '59': 'lifestyle', '60': 'we', '61': 'attended', '62': 'this', '63': 'asian', '64': 'festival', '65': 'which', '66': 'full', '67': 'color', '68': '